<a href="https://colab.research.google.com/github/MatheusFelippeMeira/AppEmpreiteiros/blob/main/RoboTrader_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import userdata
userdata.get('OPENAI_API_KEY')
import threading
import time
import random
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from tpot import TPOTClassifier
import tkinter as tk
from tkinter import messagebox, ttk
import openai
import os
import signal
import platform
import json
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import parallel_backend
import torch
import torch.nn as nn
import torch.optim as optim

# Configuração para evitar problemas de subprocessos
os.environ["LOKY_MAX_CPU_COUNT"] = "1"

# Verifica o sistema operacional
SISTEMA = platform.system()

# Configuração inicial
# Ajuste para caminho relativo ou adaptativo por sistema operacional
if SISTEMA == "Windows":
    output_dir = r"C:\\Users\\Matheus\\Desktop\\Matheus Trabalho Digital\\PoketeOption\\Modelo IA APRENDIZAGEM"
else:
    # No Linux, usamos o diretório home do usuário
    output_dir = os.path.join(os.path.expanduser("~"), "RoboTrader/modelo_ia")

os.makedirs(output_dir, exist_ok=True)
model_path = os.path.join(output_dir, "melhor_modelo.pt")

# Configuração da API OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")
if not openai.api_key:
    raise ValueError("Chave da API OpenAI não configurada. Defina a variável OPENAI_API_KEY no ambiente.")

# Definição do modelo PyTorch
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

# Configuração do modelo
input_size = 6  # Número de características
hidden_size = 64
output_size = 3  # Classes: Compre, Venda, Aguarde
model = NeuralNetwork(input_size, hidden_size, output_size)

# Verificar se o modelo salvo existe ou criar novo
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path))
    print(f"Modelo carregado de: {model_path}")
else:
    torch.save(model.state_dict(), model_path)
    print(f"Novo modelo criado e salvo em: {model_path}")

# Função para tocar som compatível com diferentes sistemas operacionais
def tocar_som(recomendacao):
    try:
        if SISTEMA == "Windows":
            import winsound
            if recomendacao == "Compre":
                winsound.Beep(1000, 500)  # Som para 'Compre'
            elif recomendacao == "Venda":
                winsound.Beep(500, 500)  # Som para 'Venda'
            elif recomendacao == "Aguarde":
                winsound.Beep(750, 300)  # Som neutro para 'Aguarde'
        elif SISTEMA == "Linux":
            # Uso de comandos de terminal para som no Linux
            freq = "1000" if recomendacao == "Compre" else "500" if recomendacao == "Venda" else "750"
            duration = "0.5" if recomendacao == "Compre" else "0.5" if recomendacao == "Venda" else "0.3"
            os.system(f"play -n synth {duration} sine {freq} 2>/dev/null || echo 'Som não suportado'")
        elif SISTEMA == "Darwin":  # macOS
            os.system(f"afplay /System/Library/Sounds/Tink.aiff")
    except Exception as e:
        print(f"Não foi possível reproduzir som: {e}")

# Classe PocketOption
class PocketOption:
    def __init__(self, ssid):
        self.ssid = ssid
        self.connected = False

    def connect(self):
        try:
            print(f"Conectando com a Pocket Option usando SSID: {self.ssid}")
            # Simulação de conexão - em ambiente real, implementar a conexão real
            self.connected = True
            return True
        except Exception as e:
            print(f"Erro ao conectar: {e}")
            self.connected = False
            return False

    def get_candles(self, ativo):
        if not self.connected:
            print("Não conectado. Reconectando...")
            self.connect()

        print(f"Obtendo candles para o ativo {ativo}...")
        candles = []
        for i in range(20):
            open_price = round(random.uniform(1.100, 1.150), 3)
            close_price = round(random.uniform(1.100, 1.150), 3)
            high_price = round(max(open_price, close_price) + random.uniform(0.001, 0.01), 3)
            low_price = round(min(open_price, close_price) - random.uniform(0.001, 0.01), 3)
            low_price = max(low_price, 0)  # Garantir que low não seja menor que zero

            candles.append({
                "timestamp": time.time() - (i * 60),
                "open": open_price,
                "close": close_price,
                "high": high_price,
                "low": low_price,
                "volume": random.randint(1000, 5000),
            })
        return candles

# Funções auxiliares de validação e análise
def validar_dados_candles(candles):
    for idx, candle in enumerate(candles):
        if candle["open"] < 0 or candle["close"] < 0 or candle["high"] < 0 or candle["low"] < 0:
            raise ValueError(f"Candle inválido (valores negativos) no índice {idx}: {candle}")
        if candle["high"] < candle["low"]:
            raise ValueError(f"Erro lógico no candle {idx}: high < low. Dados: {candle}")
    return True

def detectar_padroes(candles):
    validar_dados_candles(candles)

    if detectar_three_white_soldiers(candles):
        return "three_white_soldiers"
    if detectar_three_black_crows(candles):
        return "three_black_crows"
    if detectar_morning_star(candles):
        return "morning_star"
    if detectar_evening_star(candles):
        return "evening_star"
    if detectar_piercing_line(candles):
        return "piercing_line"

    return None

def detectar_three_white_soldiers(candles):
    if len(candles) < 3:
        return False
    c1, c2, c3 = candles[-3:]
    return (
        c1["close"] > c1["open"] and c2["close"] > c2["open"] and c3["close"] > c3["open"]
        and c1["close"] < c2["open"] and c2["close"] < c3["open"]
        and abs(c1["close"] - c1["open"]) > (c1["high"] - c1["low"]) * 0.5
        and abs(c2["close"] - c2["open"]) > (c2["high"] - c2["low"]) * 0.5
        and abs(c3["close"] - c3["open"]) > (c3["high"] - c3["low"]) * 0.5
    )

def detectar_three_black_crows(candles):
    if len(candles) < 3:
        return False
    c1, c2, c3 = candles[-3:]
    return (
        c1["close"] < c1["open"] and c2["close"] < c2["open"] and c3["close"] < c3["open"]
        and c1["close"] > c2["open"] and c2["close"] > c3["open"]
        and abs(c1["close"] - c1["open"]) > (c1["high"] - c1["low"]) * 0.5
        and abs(c2["close"] - c2["open"]) > (c2["high"] - c2["low"]) * 0.5
        and abs(c3["close"] - c3["open"]) > (c3["high"] - c3["low"]) * 0.5
    )

def detectar_morning_star(candles):
    if len(candles) < 3:
        return False
    c1, c2, c3 = candles[-3:]
    return (
        c1["close"] < c1["open"]
        and abs(c2["close"] - c2["open"]) < (c2["high"] - c2["low"]) * 0.2
        and c3["close"] > c3["open"] and c3["close"] > c1["open"]
    )

def detectar_evening_star(candles):
    if len(candles) < 3:
        return False
    c1, c2, c3 = candles[-3:]
    return (
        c1["close"] > c1["open"]
        and abs(c2["close"] - c2["open"]) < (c2["high"] - c2["low"]) * 0.2
        and c3["close"] < c3["open"] and c3["close"] < c1["open"]
    )

def detectar_piercing_line(candles):
    if len(candles) < 2:
        return False
    c1, c2 = candles[-2:]
    return (
        c1["close"] < c1["open"]
        and c2["open"] < c1["close"]
        and c2["close"] > (c1["open"] + c1["close"]) / 2
    )

def calcular_topos_fundos(candles):
    highs = [c["high"] for c in candles]
    lows = [c["low"] for c in candles]
    return max(highs), min(lows)

def calcular_momentum(candles):
    tamanhos = [abs(c["close"] - c["open"]) for c in candles]
    return {
        "current_candle_size": tamanhos[-1],
        "average_candle_size": np.mean(tamanhos),
    }

def calcular_volatilidade(candles):
    highs = np.array([c["high"] for c in candles])
    lows = np.array([c["low"] for c in candles])
    closes = np.array([c["close"] for c in candles])

    previous_closes = np.roll(closes, 1)
    previous_closes[0] = closes[0]

    tr = np.maximum(highs - lows, np.maximum(abs(highs - previous_closes), abs(lows - previous_closes)))
    atr = np.mean(tr)
    desvio_padrao = np.std(closes)

    return {
        "atr": atr,
        "std_dev": desvio_padrao
    }

def analisar_contexto(candles):
    closes = [c["close"] for c in candles]
    sma_short = np.mean(closes[-10:])
    sma_long = np.mean(closes[-20:])
    tendencia = "alta" if sma_short > sma_long else "baixa"

    atr = calcular_volatilidade(candles)["atr"]
    lateralizado = atr < 0.01

    return {"tendencia": tendencia, "lateralizado": lateralizado}

def detectar_rompimento(candles):
    closes = [c["close"] for c in candles]
    highs = [c["high"] for c in candles]
    lows = [c["low"] for c in candles]

    suporte = min(lows[-5:])
    resistencia = max(highs[-5:])
    preco_atual = closes[-1]

    rompimento_suporte = preco_atual < suporte
    rompimento_resistencia = preco_atual > resistencia

    return {"rompeu_suporte": rompimento_suporte, "rompeu_resistencia": rompimento_resistencia}

def gerar_dados(candles):
    topos_fundos = calcular_topos_fundos(candles)
    padrao = detectar_padroes(candles)
    momentum = calcular_momentum(candles)
    volatilidade = calcular_volatilidade(candles)

    return {
        "candles": candles,
        "topos_fundos": topos_fundos,
        "patterns": padrao,
        "momentum": momentum,
        "volatility": volatilidade,
    }

def usar_rede_neural_pytorch(dados):
    model.eval()
    with torch.no_grad():
        caracteristicas = torch.tensor([
            [
                dados["topos_fundos"][0],
                dados["topos_fundos"][1],
                dados["momentum"]["current_candle_size"],
                dados["momentum"]["average_candle_size"],
                dados["volatility"]["atr"],
                dados["volatility"]["std_dev"]
            ]
        ], dtype=torch.float32)

        predicao = model(caracteristicas)
        classe_predita = torch.argmax(predicao, dim=1).item()

        if classe_predita == 0:
            return "Compre"
        elif classe_predita == 1:
            return "Venda"
        else:
            return "Aguarde"

def consultar_chatgpt(dados):
    mensagem = (
        f"Analise os seguintes dados de price action e forneça uma recomendação de compra, venda ou espera:\n"
        f"- Topo máximo e fundo mínimo: {dados['topos_fundos']}\n"
        f"- Padrão detectado: {dados['patterns']}\n"
        f"- Momentum: {dados['momentum']}\n"
        f"- Volatilidade (ATR e desvio padrão): {dados['volatility']}\n"
        f"- Contexto de mercado: {analisar_contexto(dados['candles'])}\n"
        f"- Rompimento de suporte ou resistência: {detectar_rompimento(dados['candles'])}\n"
    )

    try:
        resposta = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Você é um especialista em análise técnica de price action."},
                {"role": "user", "content": mensagem}
            ]
        )
        conteudo = resposta["choices"][0]["message"]["content"].strip().lower()

        if "compre" in conteudo:
            return "Compre"
        elif "venda" in conteudo:
            return "Venda"
        else:
            return "Aguarde"
    except Exception as e:
        print(f"Erro ao consultar OpenAI: {e}")
        return usar_rede_neural_pytorch(dados)

def pre_analise(dados):
    contexto = analisar_contexto(dados["candles"])
    rompimento = detectar_rompimento(dados["candles"])
    topo, fundo = dados["topos_fundos"]
    padrao_detectado = dados["patterns"] is not None
    momentum_significativo = dados["momentum"]["current_candle_size"] > 1.35 * dados["momentum"]["average_candle_size"]
    volatilidade_adequada = dados["volatility"]["atr"] > 0.015

    pontuacao = 0
    if padrao_detectado:
        pontuacao += 1.2
    if momentum_significativo:
        pontuacao += 1
    if volatilidade_adequada:
        pontuacao += 0.7
    if contexto["tendencia"] == "alta" and not contexto["lateralizado"]:
        pontuacao += 1
    if rompimento["rompeu_resistencia"]:
        pontuacao += 1
    elif rompimento["rompeu_suporte"]:
        pontuacao -= 1

    print(f"Pontuação final da pré-análise: {pontuacao}")
    return pontuacao >= 2.5

# Nova classe para IA supervisora
class SupervisorIA:
    def __init__(self, log_path=None):
        """
        Inicializa a IA supervisora que monitora o sistema principal

        :param log_path: Caminho para salvar os logs. Se None, usa o diretório padrão
        """
        if log_path is None:
            if SISTEMA == "Windows":
                self.log_path = os.path.join(output_dir, "supervisor_logs")
            else:
                self.log_path = os.path.join(os.path.expanduser("~"), "RoboTrader/supervisor_logs")
        else:
            self.log_path = log_path

        os.makedirs(self.log_path, exist_ok=True)

        # Modelo de detecção de anomalias
        self.anomaly_detector = IsolationForest(contamination=0.05, random_state=42)

        # Histórico de recomendações para análise
        self.historico_recomendacoes = []

        # Métricas de confiabilidade
        self.metricas = {
            "total_analises": 0,
            "inconsistencias": 0,
            "anomalias_detectadas": 0,
            "ultima_verificacao": None
        }

        # Histórico de desempenho do sistema principal
        self.historico_desempenho = []

        # Verificar se há arquivo de histórico e carregar
        self.arquivo_historico = os.path.join(self.log_path, "historico_supervisor.json")
        self.carregar_historico()

        print(f"Supervisor IA iniciado. Logs salvos em: {self.log_path}")

    def carregar_historico(self):
        """Carrega o histórico de dados de um arquivo JSON se existir"""
        try:
            if os.path.exists(self.arquivo_historico):
                with open(self.arquivo_historico, 'r') as f:
                    dados = json.load(f)
                    self.historico_recomendacoes = dados.get('historico_recomendacoes', [])
                    self.metricas = dados.get('metricas', self.metricas)
                    self.historico_desempenho = dados.get('historico_desempenho', [])
                print(f"Histórico carregado: {len(self.historico_recomendacoes)} registros")
        except Exception as e:
            print(f"Erro ao carregar histórico: {e}")

    def salvar_historico(self):
        """Salva o histórico de dados em um arquivo JSON"""
        try:
            dados = {
                'historico_recomendacoes': self.historico_recomendacoes,
                'metricas': self.metricas,
                'historico_desempenho': self.historico_desempenho
            }
            with open(self.arquivo_historico, 'w') as f:
                json.dump(dados, f)
        except Exception as e:
            print(f"Erro ao salvar histórico: {e}")

    def analisar_decisao(self, dados, recomendacao_principal, recomendacao_secundaria=None):
        """
        Analisa a decisão tomada pelo sistema principal

        :param dados: Dados usados para a análise
        :param recomendacao_principal: Recomendação do sistema principal (ChatGPT)
        :param recomendacao_secundaria: Recomendação secundária (Rede Neural)
        :return: Tupla (aprovado, confianca, feedback)
        """
        # Registrar a recomendação no histórico
        timestamp = datetime.now().isoformat()

        registro = {
            'timestamp': timestamp,
            'dados': {
                'topos_fundos': dados['topos_fundos'],
                'patterns': dados['patterns'],
                'momentum_atual': dados['momentum']['current_candle_size'],
                'momentum_medio': dados['momentum']['average_candle_size'],
                'atr': dados['volatility']['atr'],
                'std_dev': dados['volatility']['std_dev']
            },
            'recomendacao_principal': recomendacao_principal
        }

        if recomendacao_secundaria:
            registro['recomendacao_secundaria'] = recomendacao_secundaria

        self.historico_recomendacoes.append(registro)
        self.metricas['total_analises'] += 1

        # Verificar inconsistências entre recomendações
        inconsistencia = False
        if recomendacao_secundaria and recomendacao_principal != recomendacao_secundaria:
            inconsistencia = True
            self.metricas['inconsistencias'] += 1

        # Analisar anomalias nos dados
        anomalia = self.verificar_anomalias(dados)
        if anomalia:
            self.metricas['anomalias_detectadas'] += 1

        # Calcular confiança na recomendação
        confianca = self.calcular_confianca(dados, inconsistencia, anomalia)

        # Gerar feedback
        feedback = self.gerar_feedback(dados, recomendacao_principal,
                                       recomendacao_secundaria, confianca)

        # Atualizar métricas
        self.metricas['ultima_verificacao'] = timestamp

        # Salvar histórico periodicamente
        if len(self.historico_recomendacoes) % 10 == 0:
            self.salvar_historico()

        # Decisão final
        aprovado = confianca >= 0.7

        return aprovado, confianca, feedback

    def verificar_anomalias(self, dados):
        """Verifica se há anomalias nos dados de entrada"""
        # Se temos poucos dados, treinar o modelo primeiro
        if len(self.historico_recomendacoes) >= 10:
            # Extrair características de dados históricos para treinar
            X_historico = []
            for reg in self.historico_recomendacoes[-50:]:  # Usamos os últimos 50 registros
                X_historico.append([
                    reg['dados']['topos_fundos'][0],
                    reg['dados']['topos_fundos'][1],
                    reg['dados']['momentum_atual'],
                    reg['dados']['momentum_medio'],
                    reg['dados']['atr'],
                    reg['dados']['std_dev']
                ])

            # Treinar o detector de anomalias
            if len(X_historico) > 5:  # Precisamos de pelo menos alguns exemplos
                self.anomaly_detector.fit(X_historico)

                # Verificar o dado atual
                X_atual = [[
                    dados['topos_fundos'][0],
                    dados['topos_fundos'][1],
                    dados['momentum']['current_candle_size'],
                    dados['momentum']['average_candle_size'],
                    dados['volatility']['atr'],
                    dados['volatility']['std_dev']
                ]]

                # -1 para anomalias, 1 para dados normais
                resultado = self.anomaly_detector.predict(X_atual)
                return resultado[0] == -1

        return False

    def calcular_confianca(self, dados, inconsistencia, anomalia):
        """Calcula um valor de confiança para a recomendação"""
        # Base de confiança
        confianca = 0.8

        # Reduzir confiança se houver inconsistência nas recomendações
        if inconsistencia:
            confianca -= 0.3

        # Reduzir confiança se houver anomalias nos dados
        if anomalia:
            confianca -= 0.4

        # Ajustar confiança com base nas características dos dados
        # Padrões de candlestick conhecidos aumentam a confiança
        if dados['patterns'] is not None:
            confianca += 0.1

        # Momentum significativo
        if dados['momentum']['current_candle_size'] > 1.5 * dados['momentum']['average_candle_size']:
            confianca += 0.05

        # Alta volatilidade reduz levemente a confiança (mais incerteza)
        if dados['volatility']['atr'] > 0.02:
            confianca -= 0.05

        # Garantir que a confiança esteja entre 0 e 1
        return max(0.0, min(1.0, confianca))

    def gerar_feedback(self, dados, recomendacao_principal, recomendacao_secundaria, confianca):
        """Gera feedback explicativo sobre a análise"""
        feedback = []

        # Analisar contexto de mercado
        contexto = analisar_contexto(dados['candles'])
        tendencia = contexto['tendencia']
        lateralizado = contexto['lateralizado']

        # Verificar se a recomendação está alinhada com a tendência
        alinhado_tendencia = (
            (tendencia == "alta" and recomendacao_principal == "Compre") or
            (tendencia == "baixa" and recomendacao_principal == "Venda")
        )

        # Comentários sobre confiança
        if confianca > 0.9:
            feedback.append("Alta confiança na recomendação.")
        elif confianca < 0.6:
            feedback.append("Baixa confiança, considere aguardar.")

        # Comentários sobre inconsistências
        if recomendacao_secundaria and recomendacao_principal != recomendacao_secundaria:
            feedback.append(f"Inconsistência entre análises: IA principal sugere {recomendacao_principal}, rede neural sugere {recomendacao_secundaria}.")

        # Comentários sobre tendência
        if not alinhado_tendencia and recomendacao_principal != "Aguarde":
            feedback.append(f"Recomendação contrária à tendência de {tendencia}.")

        # Comentários sobre padrões
        if dados['patterns']:
            feedback.append(f"Padrão {dados['patterns']} detectado, aumentando confiabilidade.")

        # Comentários sobre volatilidade
        if dados['volatility']['atr'] > 0.025:
            feedback.append("Alta volatilidade detectada, considere reduzir exposição.")

        # Comentários sobre movimento lateral
        if lateralizado and recomendacao_principal != "Aguarde":
            feedback.append("Mercado lateralizado, recomendação pode ser prematura.")

        return " ".join(feedback)

    def obter_estatisticas(self):
        """Retorna estatísticas sobre o desempenho do supervisor"""
        taxa_inconsistencia = 0
        if self.metricas['total_analises'] > 0:
            taxa_inconsistencia = self.metricas['inconsistencias'] / self.metricas['total_analises']

        return {
            "total_analises": self.metricas['total_analises'],
            "taxa_inconsistencia": f"{taxa_inconsistencia:.2%}",
            "anomalias_detectadas": self.metricas['anomalias_detectadas'],
            "ultima_verificacao": self.metricas['ultima_verificacao']
        }

def atualizar_recomendacao_thread(ativo):
    try:
        candles = pocket_option.get_candles(ativo)
        dados = gerar_dados(candles)

        if pre_analise(dados):
            recomendacao_principal = consultar_chatgpt(dados)
            # Obter segunda opinião da rede neural
            recomendacao_secundaria = usar_rede_neural_pytorch(dados)

            # Supervisão da IA
            aprovado, confianca, feedback = supervisor.analisar_decisao(
                dados, recomendacao_principal, recomendacao_secundaria)

            # Formatação da confiança
            confianca_str = f"{confianca:.0%}"

            # Decidir se aceita a recomendação com base na aprovação do supervisor
            if not aprovado:
                recomendacao = "Aguarde"
                texto_recomendacao = f"Recomendação: {recomendacao} (confiança insuficiente: {confianca_str})"
            else:
                recomendacao = recomendacao_principal
                texto_recomendacao = f"Recomendação: {recomendacao} (confiança: {confianca_str})"

            labels_recomendacoes[ativo].config(text=texto_recomendacao)

            # Atualizar o texto de feedback
            if ativo in labels_feedback:
                labels_feedback[ativo].config(text=f"Análise: {feedback}")
        else:
            recomendacao = "Aguarde"
            labels_recomendacoes[ativo].config(text=f"Recomendação: {recomendacao}")
            if ativo in labels_feedback:
                labels_feedback[ativo].config(text="Análise: Pré-análise indica espera.")

        tocar_som(recomendacao)

        # Atualiza o status para "Aguardando" após 30 segundos
        root.after(30000, lambda: labels_recomendacoes[ativo].config(text="Recomendação: Aguardando..."))
    except Exception as e:
        labels_recomendacoes[ativo].config(text=f"Erro: {str(e)[:30]}...")
        label_status.config(text="Status: Erro de conexão", fg="red")
        # Tentar reconectar em caso de falha
        try:
            pocket_option.connect()
            if pocket_option.connected:
                label_status.config(text="Status: Reconectado", fg="green")
        except:
            pass

def criar_interface():
    root = tk.Tk()
    root.title("Análise de Mercado - RoboTrader com IA Supervisora")

    # Configuração de estilo
    style = ttk.Style()
    if SISTEMA != "Windows":  # Tema específico para Linux/Mac
        try:
            style.theme_use('clam')
        except:
            pass

    # Configurar cores e estilos
    bg_color = "#f0f0f0"
    root.configure(bg=bg_color)

    # Criar notebook para abas
    notebook = ttk.Notebook(root)
    notebook.pack(fill='both', expand=True, padx=10, pady=10)

    # Aba principal para ativos
    tab_ativos = ttk.Frame(notebook)
    notebook.add(tab_ativos, text="Ativos")

    # Aba para estatísticas e supervisão
    tab_supervisor = ttk.Frame(notebook)
    notebook.add(tab_supervisor, text="Supervisão IA")

    # Controles na aba principal
    ativos = ["Boeing Company OTC", "Apple Inc OTC", "American Express OTC"]
    global labels_recomendacoes, labels_feedback
    labels_recomendacoes = {}
    labels_feedback = {}

    # Frame para exibir o status da conexão
    frame_status = ttk.Frame(tab_ativos)
    frame_status.pack(pady=5, fill='x')
    global label_status
    label_status = ttk.Label(frame_status, text="Status: Conectado", foreground="green")
    label_status.pack()

    for ativo in ativos:
        frame = ttk.LabelFrame(tab_ativos, text=ativo)
        frame.pack(pady=10, padx=10, fill='x')

        label_recomendacao = ttk.Label(frame, text="Recomendação: Aguardando...", font=("Arial", 12))
        label_recomendacao.pack(pady=5)
        labels_recomendacoes[ativo] = label_recomendacao

        # Adiciona label para feedback do supervisor
        label_feedback = ttk.Label(frame, text="Análise: Aguardando...", wraplength=350)
        label_feedback.pack(pady=5)
        labels_feedback[ativo] = label_feedback

    # Botão para forçar atualização
    def forcar_atualizacao():
        atualizar_todos_ativos()

    btn_frame = ttk.Frame(tab_ativos)
    btn_frame.pack(pady=10)
    btn_atualizar = ttk.Button(btn_frame, text="Atualizar Agora", command=forcar_atualizacao)
    btn_atualizar.pack(side=tk.LEFT, padx=5)

    # Botão para abrir configurações
    btn_config = ttk.Button(btn_frame, text="Configurações",
                           command=lambda: messagebox.showinfo("Configurações",
                                                              "Funcionalidade de configurações em desenvolvimento."))
    btn_config.pack(side=tk.LEFT, padx=5)

    # Conteúdo da aba de supervisão
    supervisor_frame = ttk.LabelFrame(tab_supervisor, text="Estatísticas da IA Supervisora")
    supervisor_frame.pack(pady=10, padx=10, fill='both', expand=True)

    # Labels para estatísticas
    labels_estatisticas = {}
    for stat in ["Total de análises", "Taxa de inconsistência", "Anomalias detectadas", "Última verificação"]:
        frame = ttk.Frame(supervisor_frame)
        frame.pack(pady=5, fill='x')

        ttk.Label(frame, text=f"{stat}:", width=20).pack(side=tk.LEFT)
        label_valor = ttk.Label(frame, text="Carregando...")
        label_valor.pack(side=tk.LEFT)
        labels_estatisticas[stat] = label_valor

    # Função para atualizar estatísticas
    def atualizar_estatisticas():
        estat = supervisor.obter_estatisticas()
        labels_estatisticas["Total de análises"].config(text=str(estat["total_analises"]))
        labels_estatisticas["Taxa de inconsistência"].config(text=estat["taxa_inconsistencia"])
        labels_estatisticas["Anomalias detectadas"].config(text=str(estat["anomalias_detectadas"]))
        labels_estatisticas["Última verificação"].config(
            text=estat["ultima_verificacao"] if estat["ultima_verificacao"] else "Nunca")

        # Agendar próxima atualização
        root.after(5000, atualizar_estatisticas)

    # Botões específicos da aba de supervisão
    btn_frame_sup = ttk.Frame(tab_supervisor)
    btn_frame_sup.pack(pady=10)

    ttk.Button(btn_frame_sup, text="Atualizar Estatísticas",
              command=atualizar_estatisticas).pack(side=tk.LEFT, padx=5)

    ttk.Button(btn_frame_sup, text="Exportar Dados",
              command=lambda: messagebox.showinfo("Exportar",
                                                "Os dados serão exportados para " +
                                                supervisor.log_path)).pack(side=tk.LEFT, padx=5)

    # Monitora a tecla ESC para sair
    root.bind('<Escape>', lambda e: root.destroy())

    def iniciar_thread(ativo):
        threading.Thread(target=atualizar_recomendacao_thread, args=(ativo,), daemon=True).start()

    def atualizar_todos_ativos():
        for ativo in ativos:
            iniciar_thread(ativo)

    def atualizar_periodicamente():
        atualizar_todos_ativos()
        root.after(60000, atualizar_periodicamente)

    # Iniciar atualizações
    atualizar_periodicamente()
    atualizar_estatisticas()

    # Tratamento de encerramento limpo
    def encerrar_aplicacao():
        print("Encerrando aplicação...")
        # Salvar dados do supervisor
        if 'supervisor' in globals():
            supervisor.salvar_historico()
        root.destroy()

    # Registra handler para sinais de interrupção
    if SISTEMA != "Windows":  # No Windows o comportamento é diferente
        signal.signal(signal.SIGINT, lambda sig, frame: encerrar_aplicacao())
        signal.signal(signal.SIGTERM, lambda sig, frame: encerrar_aplicacao())

    # Adiciona exibição da versão
    versao_label = ttk.Label(root, text="RoboTrader v1.2.0 com IA Supervisora", font=("Arial", 8))
    versao_label.pack(side=tk.BOTTOM, pady=5)

    return root

def main():
    global pocket_option, supervisor
    ssid = "2a231e5c0ad9e70da50416f16e45371"

    try:
        print("Iniciando RoboTrader com IA Supervisora...")

        # Inicializar a IA supervisora
        supervisor = SupervisorIA()

        pocket_option = PocketOption(ssid)
        if pocket_option.connect():
            print("Conexão estabelecida com sucesso.")
            root = criar_interface()
            root.mainloop()
        else:
            print("Erro ao conectar com a Pocket Option.")
    except KeyboardInterrupt:
        print("\nPrograma encerrado pelo usuário.")
    except Exception as e:
        print(f"Erro fatal na aplicação: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'tpot'